In [31]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import rc
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline


In [32]:
d = pd.read_csv('311_Service_Requests_from_2010_to_Present.csv',low_memory=False)

After loading data, lets look at the columns we have

In [33]:
d.columns.to_list()

['Unique Key',
 'Created Date',
 'Closed Date',
 'Agency',
 'Agency Name',
 'Complaint Type',
 'Descriptor',
 'Location Type',
 'Incident Zip',
 'Incident Address',
 'Street Name',
 'Cross Street 1',
 'Cross Street 2',
 'Intersection Street 1',
 'Intersection Street 2',
 'Address Type',
 'City',
 'Landmark',
 'Facility Type',
 'Status',
 'Due Date',
 'Resolution Description',
 'Resolution Action Updated Date',
 'Community Board',
 'BBL',
 'Borough',
 'X Coordinate (State Plane)',
 'Y Coordinate (State Plane)',
 'Open Data Channel Type',
 'Park Facility Name',
 'Park Borough',
 'Vehicle Type',
 'Taxi Company Borough',
 'Taxi Pick Up Location',
 'Bridge Highway Name',
 'Bridge Highway Direction',
 'Road Ramp',
 'Bridge Highway Segment',
 'Latitude',
 'Longitude',
 'Location']

Let's make sure that we have one row for each unique event.

In [34]:
len(d) - len(d['Unique Key'].unique())

0

Confirm that we downloaded only the September data

In [ ]:
print('First date in db ' + min(d['Created Date']))
print('Last date in db ' + max(d['Created Date']))
d['Date'] = pd.to_datetime(d['Created Date']).dt.date

First date in db 09/01/2021 01:00:26 AM
Last date in db 09/29/2021 12:59:56 AM


Take a look at the most reported and least reported Complaint Types for September

In [ ]:
summary_of_complaint_types = d['Complaint Type'].value_counts().rename_axis('unique_values').reset_index()
summary_of_complaint_types

The complaint type of "Radioactive Material" is rather suspicious, let's see what and when that happened

In [ ]:
d[d['Complaint Type'] == 'Radioactive Material'][['Descriptor','Date']]

How are the volume of complaints distributed throughout the Boroughs?

In [ ]:
d['Borough'].value_counts()

How are the volume of complaints distributed throughout the dates of the month?

In [ ]:
d.groupby(['Date']).size()

Create a pivot table of counts of complaints between boroughs and agency

In [ ]:

piv = pd.pivot_table(d, index='Borough',columns='Agency',values='Unique Key',aggfunc= lambda x: len(x))
piv = piv.fillna(0)
piv


In [ ]:
ui = sns.color_palette("cubehelix")
#ui = ['#7fb24d','#4ac9c9','#488cd1','#4545d9','#9143e0','#f73939','#ff9935']
sns.set(style='white', palette= sns.palplot(sns.color_palette("cubehelix")),font='sans-serif')
            #sns.palplot(sns.color_palette(flatui))
fig, axs = plt.subplots(2, 1, dpi =600,figsize=(11, 8.5));
fig.subplots_adjust(wspace=.5)
# rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
# rc('text', usetex=True)
plt.tick_params(
axis='x',          # changes apply to the x-axis
which='both',      # both major and minor ticks are affected
bottom=False,      # ticks along the bottom edge are off
top=False,         # ticks along the top edge are off
labelbottom=False) # labels along the bottom edge are off

sns.lineplot(data=d.groupby(['Date']).size(), x=d.groupby(['Date']).size().index,y=d.groupby(['Date']).size(), ax=axs[0]);
axs[0].set_xlabel('')
axs[0].set_ylabel('')
axs[0].set_xticks([])
axs[0].set_title('Borough',size = 'small', usetex = True)
axs[0].tick_params(axis='y', which='major', labelsize=8)
sns.despine(top=True, right=True, left=False, bottom=True,offset=True)


sns.countplot(y='Borough', data=d, palette = ui,orient='h',order = d['Borough'].value_counts().index, ax=axs[1]);
axs[1].set_xlabel('')
axs[1].set_ylabel('')
axs[1].set_xticks([])
axs[1].set_title('Borough',size = 'small', usetex = True)
axs[1].tick_params(axis='y', which='major', labelsize=8)
sns.despine(top=True, right=True, left=False, bottom=True,offset=True)
sumwidth = []
for p in axs[1].patches:
    axs[1].annotate(int(p.get_width()),((p.get_x() + p.get_width()), p.get_y()), xytext=(15, -8),
             fontsize=8,color='#000000',textcoords='offset points'
             ,horizontalalignment='center')
    sumwidth.append(int(p.get_width()))
sumwidth = np.sum(sumwidth)
for p in axs[1].patches:
    axs[1].annotate('{:.0f}\%'.format((int(p.get_width())/sumwidth)*100),((p.get_x() + p.get_width()), p.get_y()+.3), xytext=(15, -8),
             fontsize=8,color='#000000',textcoords='offset points'
             ,horizontalalignment='center')
#########################################################################################################################################
##########################################################################################################################################



# sns.countplot(y='Borough', data=d, palette = ui,orient='h',order = d['Borough'].value_counts().index, ax=axs[2]);
# axs[2].set_xlabel('')
# axs[2].set_ylabel('')
# axs[2].set_xticks([])
# axs[2].set_title('Borough',size = 'small', usetex = True)
# axs[2].tick_params(axis='y', which='major', labelsize=8)
# sns.despine(top=True, right=True, left=False, bottom=True,offset=True)
# sumwidth = []
# for p in axs[2].patches:
#     axs[2].annotate(int(p.get_width()),((p.get_x() + p.get_width()), p.get_y()), xytext=(15, -8),
#              fontsize=8,color='#000000',textcoords='offset points'
#              ,horizontalalignment='center')
#     sumwidth.append(int(p.get_width()))
# sumwidth = np.sum(sumwidth)
# for p in axs[2].patches:
#     axs[2].annotate('{:.0f}\%'.format((int(p.get_width())/sumwidth)*100),((p.get_x() + p.get_width()), p.get_y()+.3), xytext=(15, -8),
#              fontsize=8,color='#000000',textcoords='offset points'
#              ,horizontalalignment='center')
#########################################################################################################################################
##########################################################################################################################################
#axs[2].barh(y=d.groupby(['Borough'])['Status'].size().sort_values(ascending=True).index, width=d.groupby(['Borough'])['Status'].size().sort_values(ascending=True),); #order = d['Borough'].value_counts().index
# piv = pd.pivot_table(d, index='Borough',columns='Agency',values='Unique Key',aggfunc= lambda x: len(x))
# piv = piv.fillna(0)
# piv.plot(kind='bar',color=sns.color_palette("cubehelix").as_hex(), ax = axs[2])
# #d.groupby(['Borough','Agency'])['Agency'].size()
# axs[2].set_xlabel('')
# axs[2].set_ylabel('')
# axs[2].set_xticks([])
# axs[2].set_title('Agency',size = 'small', usetex = True)
# axs[2].tick_params(axis='y', which='major', labelsize=8)
# sns.despine(top=True, right=True, left=False, bottom=True,offset=True)
# sumwidth = []
# for p in axs[2].patches:
#     axs[2].annotate(int(p.get_width()),((p.get_x() + p.get_width()), p.get_y()), xytext=(15, -2),
#              fontsize=8,color='#000000',textcoords='offset points'
#              ,horizontalalignment='center')
#     sumwidth.append(int(p.get_width()))
# sumwidth = np.sum(sumwidth)
# for p in axs[2].patches:
#     axs[2].annotate('{:.0f}\%'.format((int(p.get_width())/sumwidth)*100),((p.get_x() + p.get_width()), p.get_y()+.3), xytext=(15, -2),
#              fontsize=8,color='#000000',textcoords='offset points'
#              ,horizontalalignment='center')
plt.savefig(R'Q:\reports_311\test.pdf')
plt.show()

In [ ]:
print('Done')